# DABN13 - Assignment 3
## Preamble: Forecasting market returns
We continue with the problem of forecasting market return that we illustrated in Lecture 5. In order to train the implementation of PCR/PLS, we will replicate our previous results with a slight twist. More specifically, we will choose tuning parameters via cross-validation. Additionally, the training sets used for model evaluation are defined differently. While the example in Lecture 5 defined an expanding window of training sets with fixed starting period, we are going to use a rolling window that moved both the start and end date of the training set.

The data for this lab is provided by two csv files. 
The file *sorted_portfolios100.csv* contains the monthly returns of 100 equally weighted portfolios sorted by size and the book-to-market ratio. The data is taken from Kenneth French's data library and missing values have been inserted. The time period covered is January 1960 until December 2009
The file *twelve_month_returns.csv* contains 12-month returns on a value-weighted market portfolio. This series takes moving 12-month sums of returns of the U.S. market factor, as provided on Kenneth French's data library. The entry in row $t$ of the dataset corresponds to the market returns over the months $t+1$ until $t+12$. Accordingly, the first observed value in our sample is the 12-month return over the period February 1960 - January 1961. The last observation covers the period January-December 2010.

To begin with the lab, we import both your outcome as well as the 100 predictors into Python using the code below. You might be required to modify the file path.

In [1]:
import numpy as np
import pandas as pd
from os import chdir

#chdir('??') Adjust directory here
portfolios  = pd.read_csv("sorted_portfolios100.csv").iloc[:,1:]
mkt_ret_12m = pd.read_csv("twelve_month_returns.csv").iloc[:,1:] 

mkt_ret_12m_train = mkt_ret_12m.iloc[:540,:]
portfolios_train  = portfolios.iloc[:540,:]

## Part 1: Setting up a PCR pipeline in `sklearn`

Principal component regression is not available in scikit-learn as a distinctive learning algorithm. This is no problem because it is the combination of three simple components:

1. Standardizing inputs,
2. obtaining $k$ principal components from the inputs,
3. running a linear regression of the output on the $k$ principal components (and a constant).

All three operations above are implemented in sklearn. In order to put them together, we will use the very convenient `Pipeline()` functionality.

Pipelines allow us to specify a custom sequence of actions. An existing pipeline can be fit, similar to the specification of a regression model. If we do that, all steps of the pipeline are executed in their specified order.

### Task 1a)

In this task, we will create and fit a pipeline for PCR. First, we need to specify the three actions mentioned above individually. This can be done by using the `StandardScalar()`, `PCA()` and `LinearRegression()` functions from several `sklearn` modules. I have already written code below to import these functions. Now, please create three objects `stdz_1a`, `PCA_1a` and `lm_1a` which specify

1. input standardization,
2. PCA with two principal components,
3. a linear regression model,

respectively.

Next, we use the `Pipeline()`-function to define the full sequence of actions that result in PCR. I already prepared the basic structure of the required line of code. All you need is to specify the three steps of our pipeline and to give them suitable names. Please do that and save the resulting object as `pcr_pipe_1a`.

Lastly, we can apply the `fit()`-method to `pcr_pipe_1a` to execute the entire pipeline. Do this using `mkt_ret_12m_train`and `portfolios_train` as your output and inputs, respectively. Save the learned model as `pcr_fit_1a`. 

In [2]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

stdz_1a = StandardScaler()      # Standardization
PCA_1a = PCA(n_components=2)    # PCA with 2 components
lm_1a = LinearRegression()      # Linear regression


pcr_pipe_1a = Pipeline(steps=[
    ('standardize', stdz_1a),
    ('pca', PCA_1a),
    ('linear_model', lm_1a)
])

pcr_fit_1a = pcr_pipe_1a.fit(portfolios_train, mkt_ret_12m_train)

pcr_fit_1a
 

Pipeline(steps=[('standardize', StandardScaler()), ('pca', PCA(n_components=2)),
                ('linear_model', LinearRegression())])

### Task 1b)

We will now get predictions from our learned principal components regression. However, we'll do that time series style. 

When we work with time series, our training data consists of historical series from some starting point in the past until today. Our goal is outcome prediction in the immediate future. That is, our test data may simply be a single data point - at least for the model that we trained at one specific point in time. 

In Task 1a) we used the earliest 540 data points, assuming that this was all our data. Now, we want to get a forecast of data point 541, assuming that this is the immediate future. The more distant future is not of immediate interest for us and we will predict it step-by-step as time passes and more training data appears. 

Now apply the `predict()` method on `pcr_fit_1a` in order to get a prediction from your model for data point 541 in `portfolios_train` and save it as `pcr_pred_1b`.

*Note:* Being able to apply `predict()` to a pipeline is very convenient because we do not have to worry about manual standardization and manual construction of the principal component scores. We simply provide the raw test data point and `sklearn` automatically conducts all necessary transformations. 

In [3]:
portfolios_test_1b = portfolios.iloc[[540], :]  # Taking the row at index 540 for prediction (indexing start=0)

# Use the predict method to forecast the next data point
pcr_pred_1b = pcr_fit_1a.predict(portfolios_test_1b)

pcr_pred_1b


array([[0.06281893]])

## Part 2: Model tuning with `sklearn`

Scikit-learn contains very flexible routines for model tuning. The tuning parameter of PCR is the number of principal components to use. We will use the PCR pipeline created in the previous part to choose it via cross-validation.

### Task 2a)

First, we need to define a grid of potential tuning parameter values. This has to be a *dictionary* object in Python which contain arrays with the candidate values for our tuning parameters. The names of the arrays must be identical to the name of the tuning parameter in the corresponding estimator function.

Given that we tune the parameters of an entire pipeline, the names of arrays must also point at which part of the pipeline the tuning parameter can be found. To give an example, assume that we want to tune an imaginary parameter `myparam` in a function that included in my pipeline under the name `myfun`. Then, the correspoding array in the dictionary object must have the name `myfun__myparam`. 

Now create a dictionary object `tune_grid_2a` that contains an array with the integers $1,2,3,4,5$ for the number of principal components to obtain in the PCA-step of your pipeline `pcr_pipe_1a`. 

In [4]:
tune_grid_2a = {
    'pca__n_components': [1, 2, 3, 4, 5]  # Tune the number of principal components in the PCA step
}

tune_grid_2a


{'pca__n_components': [1, 2, 3, 4, 5]}

### Task 2b)
Next, we must specify how exactly our cross-validation procedure should split the data into training and holdout folds. This is done by specifying a splitter class. 

Since our ultimate goal is to predict market returns in the immediate future, this goal should be mimicked by our cross-validation splits. Hence, we are using the `TimeSeriesSplit()`-function which defines many splits into historical training data and future test data.

Now use `TimeSeriesSplit()` to define an object `cv_splits_2b` which contains a specification of the desired splits. To be specific, we want the following:

- Training folds should always consist of 90 data points.
- The test fold following the training folds should only contain 1 data point.
- We want a total of 450 splits, so that the training fold window moves through the entire length of `mkt_ret_12m_train` and `portfolios_train`.
- No gap between training and test folds.

Please check the  `TimeSeriesSplit()`-[documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html#sklearn.model_selection.TimeSeriesSplit) to figure out how to set the function inputs.

In addition to that, apply the `split()`-method to `cv_splits_2b` *and* turn the resulting output into a list object that you save as `cv_splits_list_2b`. Use `portfolios` as the `X`-argument of `split()`.

Lastly, look at the items inside `cv_splits_list_2b`. They contain the indexes of data points that are part of training and hold-out folds for every split. Use the string variable `split_characterization_2b` to characterize the pattern that you see over your 450 splits.

In [5]:
from sklearn.model_selection import TimeSeriesSplit

# 1.
cv_splits_2b = TimeSeriesSplit(n_splits=450, max_train_size=90, test_size=1, gap=0)

# 2.
cv_splits_list_2b = list(cv_splits_2b.split(portfolios_train))

# 3.
cv_splits_list_2b[:3]  # Display the first 3 splits to characterize the pattern
split_characterization_2b = "It involves a rolling window where the training set consists of the most recent 90 data points, and the test set contains the next single point. With each split, both the training window and test point shift forward by one, repeating this process for 450 splits"


### Task 2c)
Tasks 1a, 2a, and 2b provide us with all the components that we need to specify and to execute a cross-valdation based tuning parameter selection.

First, use the `GridSearchCV()` function in the `model_selection` module of Scikit-learn to specify the setup for model tuning. To be more specific, we want the following:

- We want to tune the model pipeline from Task 1a (don't worry about the number of principal components specified there. It will be overwritten),
- we use the grid of tuning parameter values from Task 2a,
- cross-validation is conducted based on the splits defined in `cv_splits_2b`,
- performance should be evaluated using the (negative) root mean square error.

Look up the `GridSearchCV()`-[documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to figure out how to set the function arguments correctly and save the resulting setup for model tuning as `pcr_tune_2c`.

Second, run the tuning procedure by applying the `fit()`-method to `pcr_tune_2c`. Use `portfolios_train` and `mkt_ret_12m_train` as inputs and output, respectively.


In [6]:
from sklearn.model_selection import GridSearchCV

# 1.
pcr_tune_2c = GridSearchCV(
    estimator=pcr_pipe_1a,         # The pipeline from Task 1a
    param_grid=tune_grid_2a,       # The grid of tuning parameters from Task 2a
    cv=cv_splits_2b,               # The TimeSeriesSplit object from Task 2b
    scoring='neg_root_mean_squared_error'        # Using (negative) root mean square error as the performance metric
)

# 2.
pcr_tune_2c.fit(portfolios_train, mkt_ret_12m_train)

pcr_tune_2c.best_params_, pcr_tune_2c.best_score_




({'pca__n_components': 5}, -0.1103078052466239)

### Task 2d)

`pcr_tune_2c` contains both the optimal tuning parameter value as well as the learned model with this value.  Where inside `pcr_tune_2c` can you find these two objects? Write your specific(!) answer into the string variable `cv_results_2d`.

Additionally, extract the `cv_results_` dictionary from `pcr_tune_2c`, which emerges once we applied the `fit()`-method, into a new object called `pcr_tune_details_2d`. Where inside `pcr_tune_details_2d` can we find the estimated cross-validation estimate of test error for the different tuning parameter candidates? Write your answer into the string variable "test_err_estimates_2d".

In [7]:
# 1.
cv_results_2d = "{pcr_tune_2c.best_params_} contains the optimal tuning parameter value and {pcr_tune_2c.best_estimator_} contains the learned model."

# 2. 
pcr_tune_details_2d = pcr_tune_2c.cv_results_
test_err_estimates_2d = "The cross-validation estimates of test error (negative RMSE) are stored in pcr_tune_details_2d['mean_test_score']."

### Task 2e) 
As an alternative to PCR, we consider PLS for return forecasting. PLS is implemented in `sklearn` in the form of the `PLSRegression()` function. Now do the following:

1. Build a pipeline that first standardizes the inputs and then conducts PLS. In the PLS-part, please do not specify the number of PLS directions, but disable variable scaling. Save the pipeline as `pls_pipe_2e`.
2. Create a tuning parameter grid that contains $1,2,3,4,5$ PLS directions as candidate values. Save it as `tune_grid_2e`.
3. Use `GridSearchCV()` to choose the optiimal number of PLS directions. Use the same setup as in Task 2c. Save the resulting object as `pls_tune_2e` and don't forget to fit this object.

In [8]:
from sklearn.cross_decomposition import PLSRegression

# 1.
pls_pipe_2e = Pipeline(steps=[
    ('standardize', StandardScaler()),   # Standardization
    ('pls', PLSRegression(scale=False))  # PLS regression without variable scaling
])

# 2.
tune_grid_2e = {
    'pls__n_components': [1, 2, 3, 4, 5]  # Candidate values for number of PLS directions
}

pls_tune_2e = GridSearchCV(
    estimator=pls_pipe_2e,               # The PLS pipeline
    param_grid=tune_grid_2e,             # Tuning parameter grid
    cv=cv_splits_2b,                     # Time series cross-validation from Task 2b
    scoring='neg_root_mean_squared_error'            # Same scoring method: negative RMSE
)

pls_tune_2e.fit(portfolios_train, mkt_ret_12m_train)

pls_tune_2e.best_params_
print("Best score (negative RMSE):", pls_tune_2e.best_score_)


Best score (negative RMSE): -0.08389384819142012


### Task 2f)
Which tuning parameter choices do we get for PCR and PLS? Write your answer into the string variable `tune_best_2g`. Additionally, save the learned models with best tuning parameters as `pcr_fit_tuned_2f` and `pls_fit_tuned_2f`, respectively.

In [9]:
# Extract the best parameter choices for PCR and PLS from the respective GridSearchCV objects
tune_best_2g = (
    f"Best PCR tuning parameter: {pcr_tune_2c.best_params_}, "
    f"Best PLS tuning parameter: {pls_tune_2e.best_params_}"
)

# Extract the learned models with the best tuning parameters
pcr_fit_tuned_2f = pcr_tune_2c.best_estimator_
pls_fit_tuned_2f = pls_tune_2e.best_estimator_

tune_best_2g

"Best PCR tuning parameter: {'pca__n_components': 5}, Best PLS tuning parameter: {'pls__n_components': 5}"

## Part 3: Comparing two candidate algorithms

Part 2 helped us to find the optimal tuning parameter values for both PCR and PLS. However, which of these two (tuned) algorithms is better for forecasting returns? We will find out in this rather difficult part.

In order to choose between PCR and PLS, we check how well either of the two procedures predicts the outcomes of data points $541, 542, \ldots, 600$. We still do this in time-series fashion. That means we use data up to data point 540 to predict data point 541 and move start and end of training and hold-out data to the end of the sample one data point at a time. 
The data used to train our model is still the 90 data points prior to the test data point.

### Task 3a)

Create a vector `mkt_ret_12m_test` and a matrix `portfolios_test` that we can use for performance evaluation on the last 60 data points of `mkt_ret_12m` and `portfolios`. That is, it must go back long enough in time to include the training data for an output prediction of data point 541.

In [10]:
mkt_ret_12m_test = mkt_ret_12m.iloc[450:,:]
portfolios_test  = portfolios.iloc[450:,:]

print("Length of mkt_ret_12m_test:", len(mkt_ret_12m_test))
print("Length of portfolios_test:", len(portfolios_test))


print("mkt_ret_12m_test:\n", mkt_ret_12m_test.head())
print("portfolios_test:\n", portfolios_test.head())

Length of mkt_ret_12m_test: 150
Length of portfolios_test: 150
mkt_ret_12m_test:
             x
450  0.150247
451  0.015484
452  0.020479
453  0.127247
454  0.157453
portfolios_test:
            V2        V3        V4        V5        V6        V7        V8  \
450 -1.897447 -1.226578 -0.964487 -0.794988 -0.732212 -0.684354 -0.475344   
451 -1.962783 -1.267780 -1.011223 -0.834716 -0.772075 -0.723287 -0.499478   
452 -2.081374 -1.361274 -1.090722 -0.907412 -0.898315 -0.797082 -0.536030   
453 -2.030023 -1.306083 -1.054208 -0.887982 -0.856215 -0.783498 -0.526544   
454 -1.933433 -1.240919 -1.006789 -0.834895 -0.816393 -0.774797 -0.484628   

           V9       V10       V11  ...       V92       V93       V94  \
450 -0.332921 -0.133040  0.369844  ... -2.347556 -1.649282 -1.325922   
451 -0.348673 -0.151086  0.343901  ... -2.264339 -1.572574 -1.274359   
452 -0.426822 -0.233785  0.253739  ... -2.311235 -1.620001 -1.323670   
453 -0.417913 -0.211740  0.264922  ... -2.299125 -1.569054 -1.256

### Task 3b)

Use `cross_validate()` in sklearn to evaluate the performance of PCR and PLS with the tuning parameter values that you arrived at in Task 2f. `cross_validate()` conducts cross-validation with specified data and a desired scheme for splitting it without any grid search.

First, use `TimeSeriesSplit()`to specify splits into training and holdout folds that contain 90 and 1 data points, respectively. Save this specification as `cv_splits_3a`. The training window should move through the data points in the test data created in Task 3a one data point at a time. The holdout fold of the first split should be the 91st observation. The holdout fold of the last split should be the very last observation in the test data.

Second, use `cross_validate()` to estimate the test error of your tuned PCR and PLS algorithms from Task 2f and use root mean squared error to measure accuracy. Save the resulting objects as `pcr_eval_3b` and `pls_eval_3b`, respectively. Check the `cross_val_score()`-[documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) to get the function inputs right.

Third, check which object inside `pcr_eval_3b` or `pcr_eval_3b` contains the root mean squared errors of all holdout folds. Write your answer into the string variable `where_is_rmse_3b`. Since we only have one data point in every holdout fold, the RMSE is identical to the absolute value of our prediction error on the test fold.



In [11]:
from sklearn.model_selection import cross_validate, TimeSeriesSplit
import numpy as np

# 1. 
cv_splits_3b = TimeSeriesSplit(n_splits=60, max_train_size=90, test_size=1)


# 2.
pcr_eval_3b = cross_validate(
    pcr_fit_tuned_2f, 
    portfolios_test, 
    mkt_ret_12m_test, 
    cv=cv_splits_3b, 
    scoring='neg_root_mean_squared_error'
)

# Cross-validation for PLS
pls_eval_3b = cross_validate(
    pls_fit_tuned_2f, 
    portfolios_test, 
    mkt_ret_12m_test, 
    cv=cv_splits_3b, 
    scoring='neg_root_mean_squared_error'
)

#3.
print("PCR RMSEs:", pcr_eval_3b)
print("PLS RMSEs:", pls_eval_3b)



where_is_rmse_3b = "The root mean squared errors are stored in the 'test_score' key of the cross_validate output."


PCR RMSEs: {'fit_time': array([0.01660204, 0.01619601, 0.01576185, 0.01355314, 0.01442194,
       0.01190305, 0.00434613, 0.01047492, 0.00655413, 0.01182318,
       0.01696801, 0.00792813, 0.01224399, 0.00548697, 0.01052904,
       0.00901723, 0.00844383, 0.00680304, 0.00479889, 0.00868487,
       0.00328088, 0.00282407, 0.00306678, 0.00654221, 0.00816894,
       0.0040071 , 0.00281119, 0.00512314, 0.00270391, 0.01141524,
       0.00696921, 0.00330782, 0.00260901, 0.00734019, 0.00800395,
       0.0104928 , 0.00898409, 0.00269723, 0.00430298, 0.0037291 ,
       0.00269294, 0.00857306, 0.00264716, 0.00264597, 0.00405693,
       0.01034117, 0.00665927, 0.00359082, 0.00314188, 0.00416589,
       0.00980997, 0.00411797, 0.00253582, 0.00728297, 0.00396514,
       0.01383877, 0.00654101, 0.00278497, 0.00392914, 0.01040602]), 'score_time': array([0.00271297, 0.00123119, 0.00085521, 0.00084496, 0.00082994,
       0.00083208, 0.00078988, 0.00088787, 0.00082493, 0.00082779,
       0.0008781 , 0.0

### Task 3c)

The RMSE provided by `cross_validate()` is hard to interpret if we want to know whether any of the two machine learning algorithms is practically useful at all. A better alternative is out-of-sample R-squared. We define it mathematically as 
$$
  R^2 = 1 - \frac{ \sum_{t=t_0}^{T}(y_t - \hat{y}_t)^2}{ \sum_{t=t_0}^{T}(y_t -\bar{y}_{t,-90})^2}.
$$
Here, 

- $y_t$ is the observed output at time $t$, 
- $t_0$ is the time period of the earliest test observation (541 in our case),
- $\hat{y}_t$ is the model prediction of $y_t$ at time $t$,
- $\bar{y}_{t,-90}=90^{-1}\sum_{\ell}^{90} y_{t-\ell}$ is the average outcome of the 90 output values before $t$

To construct this R-squared, do the following

1. Construct a vector `ymeans_3c` whose element $t$ contains the average of `mkt_ret_12m` from time point $450+t$ to $539+t$. You could apply the `rolling()` method to `mkt_ret_12m_test` or write a for-loop to achieve that.
2. Extract the sum of squared prediction error of PLS and PCR over all test folds from `pcr_eval_3b` and `pls_eval_3b`. Save them as numbers `pcr_avg_errsq_3c` and `pls_avg_errsq_3c`, respectively.
4. Construct the summed squared prediction error of the historical 90-day average output but using `mkt_ret_12m_test` and `ymeans_3c`. Save the resulting number as `histavg_avg_errsq_3c`.
5. Construct out-of-sample R2 for PCR and PLS using the objects created in the previous two steps. Save them as `pcr_R2_oos_3c` and `pls_R2_oos_3c`, respectively.

Does PCR outperform PLS or is it the other way around? Does the out-of-sample R-squared suggest that any of the two methods is useful for predicting returns? Motivate your conclusion and express it in the string variable `conclusion_3c`. 

In [12]:
# 1.
ymeans_3c = mkt_ret_12m_test.rolling(window=90).mean().shift(1).iloc[90:, 0].values

# 2.
pcr_sum_errsq_3c = np.sum(np.square(pcr_eval_3b['test_score']))
pls_sum_errsq_3c = np.sum(np.square(pls_eval_3b['test_score']))


# 3.
hist_avg_errors = mkt_ret_12m_test.iloc[90:, 0].values - ymeans_3c
histavg_sum_errsq_3c = np.sum(np.square(hist_avg_errors))

# 4.
pcr_R2_oos_3c = 1 - (pcr_sum_errsq_3c / histavg_sum_errsq_3c)
pls_R2_oos_3c = 1 - (pls_sum_errsq_3c / histavg_sum_errsq_3c)


print(pcr_R2_oos_3c)
print(pls_R2_oos_3c)

conclusion_3c = "PLS outperforms PCR in predicting returns, as evidenced by the higher out-of-sample R² values (PLS: 0.8104, PCR: 0.5249). Both methods show positive and high R² values, indicating that they are very useful for predicting returns. However, the substantially higher R² for PLS suggests that it captures the underlying patterns in the data more effectively than PCR."


0.5249668399553811
0.8104508971725661


### Part 4: PCR manually

In this advanced part, we will repeat Part 1 without using the `pca()`-transformation in scikit-learn. More specifically, we will replace it with matrix operations in Numpy.

### Task 4a)

By convention, principal components are obtained from standardized data. In order to do that, we use the `StandardScaler()` function from in scikit-learn that we already made part of the PCR pipeline. 
Statistical standardization using `StandardScaler()` needs to be first specified and then fit on data. Then it can be used to transform any desired dataset. We will do this now:

1. Use `StandardScaler()` to specify and to fit an object that contains sample means and variances of each variable in `portfolios_train`. Save this object as `stdz_X_4a`.
2. Apply the `transform()`-method to `stdz_X_4a` to standardize `portfolios_train`. Save the resulting data frame as a two-dimensional *Numpy array* `X_train_stdz_4a`. 
3. An alternative method for variable scaling would be normalization. Search among the functions [in the preprocessing module](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) of `sklearn` for an equivalent to `StandardScaler()`. Then, conduct the previous two steps using this function. However, when you save objects, use the name `norm_X_4a` instead of `stdz_X_4a` and `X_train_norm_4a` instead of `X_train_stdz_4a`. 

In [13]:
# 1.
stdz_X_4a = StandardScaler().fit(portfolios_train)

# 2.
X_train_stdz_4a = stdz_X_4a.transform(portfolios_train)

# 3.
from sklearn.preprocessing import StandardScaler, MinMaxScaler
norm_X_4a = MinMaxScaler().fit(portfolios_train)

X_train_norm_4a = norm_X_4a.transform(portfolios_train)

print("Shape of standarized training data:", X_train_stdz_4a.shape)
print("Shape of normalized training data:", X_train_norm_4a.shape)



Shape of standarized training data: (540, 100)
Shape of normalized training data: (540, 100)


### Task 4b)

We can now use the eigendecomposition (a.k.a spectral decomposition) to obtain principal components (PCs). The `eig()`-function in the linear algebra module of NumPy  allows us to obtain a list object containing the eigenvectors and eigenvalues of any square matrix that we feed into `eig()`. Now, we want to construct the scores of the first two principal components. In order to do that, review the slides of Lecture 5 and do the following:

1. Use `eig()` to get the loadings of the first two principal components of `X_train_stdz_4a`. Save them as `PCloadings_4b`.
2. Use `PCloadings_4b` to construct the scores of the first two PCs of `X_train_stdz_4a` and save them as `PCscores_4b`.

In [14]:

# 1.
from numpy.linalg import eig
eigdecomp_4b  = eig(np.cov(X_train_stdz_4a, rowvar=False))
PCloadings_4b = eigdecomp_4b[1][:, :2]  # First two principal components


# 2.
PCscores_4b = X_train_stdz_4a.dot(PCloadings_4b)


print("PC Loadings (first two PCs):", PCloadings_4b)
print("PC Scores (first two PCs):", PCscores_4b)
print("Shape of PC Scores:", PCscores_4b.shape) 

PC Loadings (first two PCs): [[-0.07926133 -0.2124923 ]
 [-0.08603607 -0.20456677]
 [-0.10002694 -0.1279695 ]
 [-0.09671834 -0.13837178]
 [-0.10309734 -0.10969116]
 [-0.10513706 -0.08805433]
 [-0.10458601 -0.09612065]
 [-0.10490294 -0.10946475]
 [-0.10728467 -0.12943632]
 [-0.07597539 -0.24512581]
 [-0.08889308 -0.19269466]
 [-0.10167646 -0.1150976 ]
 [-0.1011247  -0.09226229]
 [-0.10986611 -0.03557557]
 [-0.10965348 -0.04592798]
 [-0.10929553 -0.05145185]
 [-0.10768856 -0.04981209]
 [-0.10799659 -0.0430848 ]
 [-0.10689566 -0.07711481]
 [-0.08334093 -0.1880725 ]
 [-0.09169193 -0.14636092]
 [-0.10134743 -0.0525782 ]
 [-0.10444474 -0.02404361]
 [-0.10520473 -0.02689032]
 [-0.10947391 -0.01761684]
 [-0.11060091 -0.00940878]
 [-0.10544335  0.02176844]
 [-0.10860905 -0.01724113]
 [-0.1088974  -0.05576594]
 [-0.07733543 -0.19250444]
 [-0.10047496 -0.02835878]
 [-0.09476931 -0.07460696]
 [-0.10601483  0.01404015]
 [-0.1084487   0.040879  ]
 [-0.1063019   0.00665202]
 [-0.11039002  0.01249573]

Proceed with the second step of PCR: 

1. Create an input matrix `Z_4c` containing `PCscores_4b` and a constant. 
2. Obtain the learned coefficients for a linear regression model with inputs `Z_4c`, output `mkt_ret_12m_train` and squared error loss. Do this manually using only the `solve()` and `transpose()` commands in NumPy as well as matrix operations. Save the result as `pcr_coefs_4c`.

In [15]:
# 1.
ones_column = np.ones((PCscores_4b.shape[0], 1))  # Constant (ones) column
Z_4c = np.hstack((ones_column, PCscores_4b))  # Concatenate the constant column with PCscores_4b


# 2.
# The formula for the coefficients is: (Z^T * Z)^(-1) * Z^T * y
Z_transpose = Z_4c.T
ZtZ = Z_transpose.dot(Z_4c)  # Z^T * Z
Zty = Z_transpose.dot(mkt_ret_12m_train)  # Z^T * y
pcr_coefs_4c = np.linalg.solve(ZtZ, Zty)  # (Z^T * Z)^(-1) * Z^T * y

pcr_coefs_4c

array([[ 0.10037935],
       [-0.00378116],
       [ 0.00865954]])

### Task 4c)

Now that we have learned our PCR model, we can get an output prediction for data point 541.

1. Transform the 100 portfolio returns in the 451-st data point of `portfolios` using your fitted standardization object from Task 4a. In order to prevent Python from converting this single row of `portfolios` into a Pandas series, you might add *additional* square brackets around the row that you index. Save the resulting $1\times 100$ NumPy array of standardized inputs as `X_test_4d`.
2. Use `PCloadings_4b` to get the PC scores corresponding to the 100 values in `X_test_4d`. Save them, together with a constant, in a $1\times 3$ NumPy array `Z_test_4d`.
3. Get a prediction from the model learned in Task 4c for the input combination `Z_test_4d`. Save this prediction as `pcr_pred_4d`.

In [16]:
X_test_4d = stdz_X_4a.transform(portfolios.iloc[[450]])


PCscores_test_4d = X_test_4d.dot(PCloadings_4b)


Z_test_4d = np.hstack([np.ones((1, 1)), PCscores_test_4d])  


pcr_pred_4d = Z_test_4d.dot(pcr_coefs_4c)

print("PCR Prediction for data point 541 (pcr_pred_4d):", pcr_pred_4d)

PCR Prediction for data point 541 (pcr_pred_4d): [[0.0390604]]
